In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        '''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n        "

In [2]:
from fastai.text.all import *

In [3]:
train = pd.read_csv('/kaggle/input/twitter/twitter_training.csv')
valid = pd.read_csv('/kaggle/input/twitter/twitter_validation.csv')

In [4]:
train.isnull().sum()

2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64

In [5]:
valid.isnull().sum()

3364                                                                                                                                                                                                                                                  0
Facebook                                                                                                                                                                                                                                              0
Irrelevant                                                                                                                                                                                                                                            0
I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣    0
dtype: i

In [6]:
train.columns = ["id", "info", "type", "text"]
train

,id,info,type,text
0,2401,Borderlands,Positive,"I am coming to the borders and I will kill you all,"
1,2401,Borderlands,Positive,"im getting on borderlands and i will kill you all,"
2,2401,Borderlands,Positive,"im coming on borderlands and i will murder you all,"
3,2401,Borderlands,Positive,"im getting on borderlands 2 and i will murder you me all,"
4,2401,Borderlands,Positive,"im getting into borderlands and i can murder you all,"
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my Mac is like 6 years behind Nvidia drivers and I have no idea how I did not notice
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is 6 years behind on Nvidia drivers and I have no idea how I didn't notice
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac is now 6 years behind on Nvidia drivers and I have no idea how he didn’t notice
74679,9200,Nvidia,Positive,Just realized between the windows partition of my Mac is like being 6 years behind on Nvidia drivers and cars I have no fucking idea how I ever didn ’ t notice


In [7]:
valid.columns = ["id", "info", "type", "text"]
valid

,id,info,type,text
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects claims company acted like a 'drug dealer' bbc.co.uk/news/av/busine…
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it functions so poorly on my @SamsungUS Chromebook? 🙄
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking, it's a truly awful game."
3,4433,Google,Neutral,Now the President is slapping Americans in the face that he really did commit an unlawful act after his acquittal! From Discover on Google vanityfair.com/news/2020/02/t…
4,6273,FIFA,Negative,"Hi @EAHelp I’ve had Madeleine McCann in my cellar for the past 13 years and the little sneaky thing just escaped whilst I was loading up some fifa points, she took my card and I’m having to use my paypal account but it isn’t working, can you help me resolve it please?"
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,"⭐️ Toronto is the arts and culture capital of Canada, it’s no wonder! If you want to start planning, be sure to check out our GTA Real Estate market report for Fall 2020, it has all the info you need to finally make a move! blog.remax.ca/toronto-housin… twitter.com/kevinyoufool/s…"
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VIEWERS.\n\nI was one of those people who got hooked into csgo by watching tournaments first before playing the game. And seeing these players grew is like a netflix docu series for me. Can't wait for 2021.
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play borderlands until the sun comes up so I can hate myself all day tomorrow.
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [8]:
# Fill missing values in the 'text' column with a placeholder string
train['text'] = train['text'].fillna('Missing') 


In [9]:
train.isnull().sum()

id      0
info    0
type    0
text    0
dtype: int64

In [10]:
train.columns

Index(['id', 'info', 'type', 'text'], dtype='object')

In [11]:
dls = DataBlock(
    blocks=TextBlock.from_df('text', is_lm=True),
    get_items=ColReader('text'), splitter=RandomSplitter(0.1)
).dataloaders(train, bs=64, seq_len=80)

In [12]:
dls.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj it will be a dream come true . xxbos xxup nvidia $ nvda xxmaj this xxmaj an xxmaj excellent xxbos xxmaj this looks … kinda clean ! xxbos yo fuck this i ca n't believe i just wasted 10 minutes of my life on a fucking world of warcraft cutscene fuck this piece of shitty ass shit game xxbos then why cry here about how you are treated here . if you do n't like it , go","xxmaj it will be a dream come true . xxbos xxup nvidia $ nvda xxmaj this xxmaj an xxmaj excellent xxbos xxmaj this looks … kinda clean ! xxbos yo fuck this i ca n't believe i just wasted 10 minutes of my life on a fucking world of warcraft cutscene fuck this piece of shitty ass shit game xxbos then why cry here about how you are treated here . if you do n't like it , go to"
1,"earned the [ the xxmaj second xxmaj threat ] xxmaj achievement ! xxbos xxmaj say hello to xxmaj sasha xxmaj hostyn aka xxmaj scarlett , currently the highest earning esports player in the world ! xxmaj she hails from xxmaj canada and has built her amazing career in xxmaj starcraft 2 and xxmaj dota 2 .. xxmaj much respect to this amazing woman and her role model ! . @ xxunk pic.twitter.com / dm71v11cty xxbos xxmaj it 's going to","the [ the xxmaj second xxmaj threat ] xxmaj achievement ! xxbos xxmaj say hello to xxmaj sasha xxmaj hostyn aka xxmaj scarlett , currently the highest earning esports player in the world ! xxmaj she hails from xxmaj canada and has built her amazing career in xxmaj starcraft 2 and xxmaj dota 2 .. xxmaj much respect to this amazing woman and her role model ! . @ xxunk pic.twitter.com / dm71v11cty xxbos xxmaj it 's going to be"


In [13]:
learn = language_model_learner(
    dls,AWD_LSTM,drop_mult=0.3,
metrics=[accuracy,Perplexity()]
).to_fp16()

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [14]:
learn.fit_one_cycle(1,2e-2)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.111489,3.984064,0.302163,53.734974,02:11


In [15]:
learn.save('epoch1')

Path('models/epoch1.pth')

In [16]:
learn = learn.load('epoch1')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device)


In [17]:
learn.unfreeze()
learn.fit_one_cycle(6,2e-3)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.594631,3.668375,0.343707,39.188194,02:19
1,3.163082,3.202321,0.418105,24.589531,02:27
2,2.623580,2.847234,0.487137,17.240030,02:28
3,2.160713,2.612622,0.540951,13.634761,02:31
4,1.856373,2.525700,0.566461,12.499638,02:22
5,1.670139,2.521040,0.570544,12.441525,02:35


In [18]:
learn.save_encoder('finetuned')

In [19]:
dls2 = DataBlock(blocks=(TextBlock.from_df('text',seq_len=72,vocab=dls.vocab),CategoryBlock),
                get_x=ColReader('text'),
                get_y=ColReader('type'),
                splitter=RandomSplitter(0.1)).dataloaders(train,bs=64,seq_len=72)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [20]:
dls2.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy , xxmaj italy",Irrelevant
1,xxbos # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #,Negative
2,xxbos xxmaj um - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m - m -,Irrelevant


In [21]:
learn = text_classifier_learner(dls2,AWD_LSTM,drop_mult=0.5,metrics=accuracy).to_fp16()

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [22]:
learn = learn.load_encoder('finetuned')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(join_path_file(file,self.path/self.model_

In [23]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,1.087831,1.008026,0.571237,00:36


In [24]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.985556,0.870743,0.638457,00:37


In [25]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-3/(2.6**4),1e-3))

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.918036,0.831158,0.657874,00:51
1,0.817752,0.703869,0.717461,00:51
2,0.712501,0.546219,0.786422,00:51
3,0.566191,0.417119,0.838913,00:51
4,0.526801,0.336742,0.871318,00:51
5,0.414411,0.277676,0.896224,00:50
6,0.365399,0.258102,0.899571,00:50
7,0.375236,0.245268,0.906535,00:50
8,0.307283,0.235390,0.911757,00:51
9,0.311615,0.235573,0.910819,00:50


In [26]:
learn.export('model')

In [27]:
learn = load_learner('/kaggle/working/model')

In [28]:
pred,_,_ = learn.predict('Just realized that my Mac window partition')
print(pred)


Neutral
